# Exercises
Using the repo setup directions, setup a new local and remote repository named `spark-exercises`. The local version of your repo should live inside of `~/codeup-data-science`. This repo should be named `spark-exercises`

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

In [4]:
import pyspark
import numpy as np
import pandas as pd

## 1. Create a spark data frame that contains your favorite programming languages.



- <b>The name of the column should be language</b>


In [46]:
# spark = pyspark.sql.SparkSession.builder.getOrCreate()

import multiprocessing
# import pyspark

nprocs = multiprocessing.cpu_count()

spark = (pyspark.sql.SparkSession.builder
 .master('local')
 .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.16')
 .config('spark.driver.memory', '4G')
 .config('spark.driver.cores', nprocs)
 .config('spark.sql.shuffle.partitions', nprocs)
 .appName('MySparkApplication')
 .getOrCreate())

22/10/20 16:17:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
data = pd.DataFrame({'language': ['python', 'sql', 'matplotlib', 'seaborn', 'sklearn', 'numpy', 'pandas']})

In [7]:
programs = spark.createDataFrame(data)

- **View the schema of the dataframe**


In [9]:
programs.schema

StructType([StructField('language', StringType(), True)])

- **Output the shape of the dataframe**


In [10]:
programs.count(), len(programs.columns)

(7, 1)

- **Show the first 5 records in the dataframe**

In [11]:
programs.show(5)

+----------+
|  language|
+----------+
|    python|
|       sql|
|matplotlib|
|   seaborn|
|   sklearn|
+----------+
only showing top 5 rows



## 2. Load the mpg dataset as a spark dataframe.

In [12]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



- Create 1 column of output that contains a message like the one below:


> The 1999 audi a4 has a 4 cylinder engine.

For each vehicle.

In [13]:
import pyspark.sql.functions as F
# from pyspark.sql.functions import * 

In [47]:

mpg.select(F.concat(F.lit('The '),
                     mpg.year, 
                     F.lit(' '),
                     mpg.manufacturer, 
                     F.lit(' '),
                     mpg.model,
                     F.lit(' has a '),
                     mpg.cyl, 
                     F.lit(' cylinder engine.')
                     ).alias('Cylinders')).show(truncate=False)

+--------------------------------------------------------------+
|Cylinders                                                     |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

- Transform the trans column so that it only contains either manual or auto.

In [22]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [27]:
mpg.select(regexp_extract('trans', r'^(\w+)', 1).alias('Transmission')).show()

+------------+
|Transmission|
+------------+
|        auto|
|      manual|
|      manual|
|        auto|
|        auto|
|      manual|
|        auto|
|      manual|
|        auto|
|      manual|
|        auto|
|        auto|
|      manual|
|        auto|
|      manual|
|        auto|
|        auto|
|        auto|
|        auto|
|        auto|
+------------+
only showing top 20 rows



In [82]:
mpg.withColumn('tran',
                F.when(
                    mpg.trans.like('auot%'), 'auto'
                ).otherwise('manual')).select('trans', 'tran').show()

+----------+------+
|     trans|  tran|
+----------+------+
|  auto(l5)|manual|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|manual|
|  auto(l5)|manual|
|manual(m5)|manual|
|  auto(av)|manual|
|manual(m5)|manual|
|  auto(l5)|manual|
|manual(m6)|manual|
|  auto(s6)|manual|
|  auto(l5)|manual|
|manual(m5)|manual|
|  auto(s6)|manual|
|manual(m6)|manual|
|  auto(l5)|manual|
|  auto(s6)|manual|
|  auto(s6)|manual|
|  auto(l4)|manual|
|  auto(l4)|manual|
+----------+------+
only showing top 20 rows



## 3. Load the tips dataset as a spark dataframe.



In [28]:
tips = spark.createDataFrame(data('tips'))
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

- What percentage of observations are smokers?


In [34]:
tips.groupby(tips.smoker).count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [49]:
tips.filter(tips.smoker == 'Yes').show()

+----------+----+------+------+----+------+----+
|total_bill| tip|   sex|smoker| day|  time|size|
+----------+----+------+------+----+------+----+
|     38.01| 3.0|  Male|   Yes| Sat|Dinner|   4|
|     11.24|1.76|  Male|   Yes| Sat|Dinner|   2|
|     20.29|3.21|  Male|   Yes| Sat|Dinner|   2|
|     13.81| 2.0|  Male|   Yes| Sat|Dinner|   2|
|     11.02|1.98|  Male|   Yes| Sat|Dinner|   2|
|     18.29|3.76|  Male|   Yes| Sat|Dinner|   4|
|      3.07| 1.0|Female|   Yes| Sat|Dinner|   1|
|     15.01|2.09|  Male|   Yes| Sat|Dinner|   2|
|     26.86|3.14|Female|   Yes| Sat|Dinner|   2|
|     25.28| 5.0|Female|   Yes| Sat|Dinner|   2|
|     17.92|3.08|  Male|   Yes| Sat|Dinner|   2|
|     19.44| 3.0|  Male|   Yes|Thur| Lunch|   2|
|     32.68| 5.0|  Male|   Yes|Thur| Lunch|   2|
|     28.97| 3.0|  Male|   Yes| Fri|Dinner|   2|
|      5.75| 1.0|Female|   Yes| Fri|Dinner|   2|
|     16.32| 4.3|Female|   Yes| Fri|Dinner|   2|
|     40.17|4.73|  Male|   Yes| Fri|Dinner|   4|
|     27.28| 4.0|  M

22/10/20 18:11:57 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3378720 ms exceeds timeout 120000 ms
22/10/20 18:11:57 WARN SparkContext: Killing executors is not supported by current scheduler.


In [68]:
tips.filter(tips.smoker == 'Yes').count()

93

In [70]:
tips.select(tips.smoker).count()

244

In [71]:
tips.filter(tips.smoker == 'Yes').count() / tips.select(tips.smoker).count()

0.38114754098360654

In [89]:
tips.filter(tips.smoker == 'Yes').count() / tips.select(tips.smoker).count()

0.38114754098360654

In [83]:
tips.groupby('smoker').count().withColumn('percent', F.round(F.col('count') / tips.count() * 100)).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|   62.0|
|   Yes|   93|   38.0|
+------+-----+-------+



- Create a column that contains the tip percentage


In [77]:
tips.select(F.expr('tip / total_bill AS tip_percentage')).show()

+-------------------+
|     tip_percentage|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
|0.18623962040332148|
|0.22805017103762829|
|0.11607142857142858|
|0.13031914893617022|
| 0.2185385656292287|
| 0.1665043816942551|
|0.14180374361883155|
|0.10181582360570687|
|0.16277807921866522|
|0.20364126770060686|
|0.18164967562557924|
| 0.1616650532429816|
|0.22774708410067526|
|0.20624631703005306|
|0.16222760290556903|
+-------------------+
only showing top 20 rows



- Calculate the average tip percentage for each combination of sex and smoker.

In [81]:
tips.rollup('sex', 'smoker').mean(tips.select(F.expr('tip / total_bill AS tip_percentage'))).show()

AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [ ]:
tips.rollup('sex', 'smoker').mean(F.expr('tip / total_bill AS tip_percentage')).show()

TypeError: Column is not iterable

In [90]:
tips.withColumn('tip_percentage', tips.tip / tips.total_bill).groupby('sex', 'smoker').agg(F.round(F.mean('tip_percentage'),2).alias('avg_tips')).show()

+------+------+--------+
|   sex|smoker|avg_tips|
+------+------+--------+
|  Male|    No|    0.16|
|Female|    No|    0.16|
|  Male|   Yes|    0.15|
|Female|   Yes|    0.18|
+------+------+--------+



In [ ]:
tips.groupby('sex').pivot('smoker').agg(F.round(F.mean()))

- Create a column that contains the tip percentage


- Calculate the average tip percentage for each combination of sex and smoker.

## 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.



In [91]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



- Convert the temperatures to fahrenheit.


- Which month has the most rain, on average?

In [96]:
weather.withColumn(
    'month', 
    F.month(weather.date)
).groupby(
    F.col('month')
).agg(
    F.mean(weather.precipitation).alias('avg_rainfall')).sort(F.col('avg_rainfall').desc()).first()

Row(month=11, avg_rainfall=5.354166666666667)

- Which year was the windiest?


In [94]:
weather.withColumn(
    'year',
    F.year(weather.date)
).groupby(
    F.col('year')
).agg(
    F.mean(weather.wind).alias('avg_wind')
).sort(
    F.col('avg_wind').desc()
).first()

Row(year=2012, avg_wind=3.400819672131148)

- What is the most frequent type of weather in January?


- What is the average high and low temperature on sunny days in July in 2013 and 2014?


In [ ]:
(
weather.filter(F.month('date') == 7)
.filter(F.year('date') > 2012)
.filter(F.year('date') < 2015)
.filter(F.col('weather') == 'sunny')
)

- What percentage of days were rainy in q3 of 2015?


In [ ]:
(
    weather.filter(F.year('date') == 2015)
    .filter(F.quarter('date') == 3)
    .select(
        F.when('weather') == 'rain', 1
    ).otherwise(0).alias('did_rain')
)

- For each year, find what percentage of days it rained (had non-zero precipitation).

In [97]:
(
    weather.withColumn(
        'year', F.year('date')
    )
    .select(
        F.when(
            F.col('precipitation') > 0, 1
        ).otherwise(0).alias('did_rain'), 'year'
    )
    .groupby('year')
    .agg(F.mean('did_rain'))
    .show()
)

+----+-------------------+
|year|      avg(did_rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

